In [1]:
import dask
import dask.bag as db
import dask.dataframe as dd
from dask.distributed import get_worker
from dask.distributed import Client, LocalCluster, progress


In [2]:
# cluster.close()

In [3]:
# client = Client(processes=True)
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit='4GB', processes=True)
client = Client(cluster)
# client = Client()
client

Client Scheduler: tcp://127.0.0.1:35031 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.00 GB


In [4]:
# client.close()
# cluster.close()

In [5]:
import glob
import pickle
import spacy
import pandas as pd

from wb_nlp import dir_manager
from wb_nlp.cleaning import cleaner

In [6]:
# f = [f for f in glob.glob(dir_manager.get_path_from_root('src', 'wb_nlp', 'cleaning', '*.py'))]
SOURCE_DIR = dir_manager.get_data_dir('raw', 'sample_data', 'TXT_SAMPLE')
TARGET_DIR = dir_manager.get_data_dir('preprocessed', 'sample_data', 'TXT_SAMPLE')

# Create TARGET_DIR if not available
if not os.path.isdir(TARGET_DIR):
    os.makedirs(TARGET_DIR)

f = [f for f in glob.glob(os.path.join(SOURCE_DIR, '*.txt'))]
len(f)

107

In [7]:
f[0].replace(SOURCE_DIR, TARGET_DIR)

'/workspace/data/preprocessed/sample_data/TXT_SAMPLE/wb_13720575.txt'

In [8]:
# def read_file(f):
    
#     worker = get_worker()

#     try:
#         nlp = worker.nlp
#         lda_cleaner = worker.lda_cleaner
#     except AttributeError:
#         nlp = spacy.load('en_core_web_sm')
#         lda_cleaner = cleaner.LDACleaner()
#         worker.nlp = nlp
#         worker.lda_cleaner = lda_cleaner

#     content = {}
#     with open(f) as fl:
#         tokens = lda_cleaner.get_tokens_and_phrases(fl.read())
#         content[f] = tokens
#         # doc = nlp(fl.read())
#         # content[f] = [token.lemma_ for token in doc]
#         with open(f + '.txt', 'w') as ff:
#             fl.seek(0)
#             ff.write(fl.read())

#     # return True

#     return content


def clean_file(f):

    worker = get_worker()

    try:
        nlp = worker.nlp
        lda_cleaner = worker.lda_cleaner
    except AttributeError:
        nlp = spacy.load('en_core_web_sm')
        lda_cleaner = cleaner.LDACleaner()
        worker.nlp = nlp
        worker.lda_cleaner = lda_cleaner

    # content = {}
    with open(f, 'rb') as fl:
        text = fl.read().decode('utf-8', errors='ignore')
        tokens = lda_cleaner.get_tokens_and_phrases(text)
        # content[f] = tokens

        f = f.replace(SOURCE_DIR, TARGET_DIR)

        with open(f + '.pickle', 'wb') as ff:
            pickle.dump(tokens, ff)

        # doc = nlp(fl.read())
        # content[f] = [token.lemma_ for token in doc]
        # with open(f + '.txt', 'w') as ff:
        #     fl.seek(0)
        #     ff.write(fl.read())

    return True

    # return content

In [9]:
b = db.from_sequence(f, npartitions=10)
file_reader = b.map(clean_file)

In [10]:
%%time
res = file_reader.all().compute()
# res = file_reader.compute()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
CPU times: user 45.2 s, sys: 11.9 s, total: 57.1 s
Wall time: 9min 17s


In [11]:
with open(os.path.join(TARGET_DIR, 'wb_725385.txt.pickle'), 'rb') as ff:
    data = pickle.load(ff)

In [15]:
len(data['tokens']), len(data['phrases'])

(45512, 13294)

In [17]:
# data['tokens']

In [14]:
client.cancel(b)

In [15]:
client.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Client Scheduler: tcp://127.0.0.1:34733 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [12]:
%%time
res = file_reader.compute()

CPU times: user 718 ms, sys: 446 ms, total: 1.16 s
Wall time: 5.85 s


In [14]:
%%time
res = file_reader.compute()

CPU times: user 5.37 s, sys: 3.42 s, total: 8.79 s
Wall time: 50.7 s
